In [2]:
import numpy as np 
import pandas as pd 
!pip install mlxtend
from mlxtend.frequent_patterns import apriori, association_rules 


     |████████████████████████████████| 1.3MB 3.6MB/s eta 0:00:01
     |████████████████████████████████| 10.5MB 96kB/s  eta 0:00:01
     |████████████████████████████████| 25.2MB 45kB/s  eta 0:00:01
ERROR: scikit-image 0.15.0 has requirement matplotlib!=3.0.0,>=2.0.0, but you'll have matplotlib 3.0.0 which is incompatible.
ERROR: azureml-train-automl 1.0.48 has requirement azureml-telemetry==1.0.48.*, but you'll have azureml-telemetry 1.0.45 which is incompatible.
ERROR: azureml-train-automl 1.0.48 has requirement pandas<=0.23.4,>=0.21.0, but you'll have pandas 0.25.0 which is incompatible.
ERROR: azureml-train-automl 1.0.48 has requirement scipy<=1.1.0,>=1.0.0, but you'll have scipy 1.3.0 which is incompatible.
ERROR: azureml-automl-core 1.0.48 has requirement pandas<=0.23.4,>=0.21.0, but you'll have pandas 0.25.0 which is incompatible.
ERROR: azureml-automl-core 1.0.48 has requirement scipy<=1.1.0,>=1.0.0, but you'll have scipy 1.3.0 which is incompatible.
  Found existing installat

In [10]:
cd C:\Users\Dev\Desktop\Kaggle\Apriori Algorithm 
data = pd.read_excel('Online Retail.xlsx') 
data.head() 

SyntaxError: invalid syntax (<ipython-input-10-7f75872eabf9>, line 1)

In [11]:
data.Country.unique() 

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [12]:
data['Description'] = data['Description'].str.strip() 
  
# Dropping the rows without any invoice number 
data.dropna(axis = 0, subset =['InvoiceNo'], inplace = True) 
data['InvoiceNo'] = data['InvoiceNo'].astype('str') 
  
# Dropping all transactions which were done on credit 
data = data[~data['InvoiceNo'].str.contains('C')] 

In [13]:
basket_France = (data[data['Country'] =="France"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 
  
# Transactions done in the United Kingdom 
basket_UK = (data[data['Country'] =="United Kingdom"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 
  
# Transactions done in Portugal 
basket_Por = (data[data['Country'] =="Portugal"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 
  
basket_Sweden = (data[data['Country'] =="Sweden"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 

In [14]:
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1
  
# Encoding the datasets 
basket_encoded = basket_France.applymap(hot_encode) 
basket_France = basket_encoded 
  
basket_encoded = basket_UK.applymap(hot_encode) 
basket_UK = basket_encoded 
  
basket_encoded = basket_Por.applymap(hot_encode) 
basket_Por = basket_encoded 
  
basket_encoded = basket_Sweden.applymap(hot_encode) 
basket_Sweden = basket_encoded 

In [15]:
frq_items = apriori(basket_France, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
print(rules.head())

                                           antecedents  \
44                        (JUMBO BAG WOODLAND ANIMALS)   
259  (RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...   
271  (RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...   
300  (SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...   
301  (SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...   

                         consequents  antecedent support  consequent support  \
44                         (POSTAGE)            0.076531            0.765306   
259                        (POSTAGE)            0.051020            0.765306   
271                        (POSTAGE)            0.053571            0.765306   
300  (SET/6 RED SPOTTY PAPER PLATES)            0.102041            0.127551   
301    (SET/6 RED SPOTTY PAPER CUPS)            0.102041            0.137755   

      support  confidence      lift  leverage  conviction  
44   0.076531       1.000  1.306667  0.017961         inf  
259  0.051020       1.000  1.306667  0.011974     

In [16]:
frq_items = apriori(basket_UK, min_support = 0.01, use_colnames = True) 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
print(rules.head()) 

MemoryError: 